In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
housetrain=read.csv("../input/house-prices-advanced-regression-techniques/train.csv")

In [ ]:
housetest= read.csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
head(housetrain)

In [ ]:
head((housetest))

In [ ]:
str(housetrain)

In [ ]:
sort(colSums(is.na(housetrain)),decreasing = T)

In [ ]:
combinedf =rbind(housetrain[,-81],housetest)

In [ ]:
sort(colSums(is.na(combinedf)), decreasing = T)

In [ ]:
missingcols=c("PoolQC","MiscFeature","Alley","Fence","FireplaceQu")

for (col in missingcols){
    combinedf[,col]=ifelse(is.na(combinedf[,col]),"missing", combinedf[,col])
}

In [ ]:
library(tidyverse)

In [ ]:
charactercols=select_if(combinedf,is.character)
numericols=select_if(combinedf,is.numeric)

In [ ]:
str(charactercols)

In [ ]:
colnames(numericols)

In [ ]:
categorycols=numericols[,c(5,6,7,8,26,36,37)]

In [ ]:
numericols=numericols[,-c(5,6,7,8,26,36,37)]

In [ ]:
sort(colSums(is.na(charactercols)),decreasing=T)

In [ ]:
for (col in colnames(charactercols)){
    charactercols[,col]=ifelse(is.na(charactercols[,col]),which.max(table(charactercols[,col])),charactercols[,col])
}

In [ ]:
sort(colSums(is.na(numericols)),decreasing=T)

In [ ]:
for (col in colnames(numericols)){
    numericols[,col]=ifelse(is.na(numericols[,col]),median(numericols[,col],na.rm=T),numericols[,col])
}

In [ ]:
sort(colSums(is.na(categorycols)),decreasing=T)

In [ ]:
categorycols$GarageYrBlt=ifelse(is.na(categorycols$GarageYrBlt),median(categorycols$GarageYrBlt,na.rm=T),categorycols$GarageYrBlt)

In [ ]:
charactercols=lapply(charactercols,factor)
charactercols=data.frame(charactercols)

In [ ]:
categorycols=lapply(categorycols,factor)
categorycols=data.frame(categorycols)

In [ ]:
combinedfclean=cbind(numericols,charactercols,categorycols)

In [ ]:
housetraindf=combinedfclean[1:1460,]
housetestdf=combinedfclean[1461:2919,]

In [ ]:
housetraindf$SalePrice=housetrain$SalePrice

In [ ]:
colnames(housetraindf)

In [ ]:
hist(housetraindf$SalePrice)

In [ ]:
boxplot(housetraindf$SalePrice,horizontal = T)

In [ ]:
plot(density(housetraindf$SalePrice))

In [ ]:
reg=lm(SalePrice~.,data=housetraindf[-1])

In [ ]:
summary(reg)

In [ ]:
regresidual=housetraindf$SalePrice-reg$fitted.values

In [ ]:
sqrt(mean(reg$residuals^2))

In [ ]:
library(rpart)
houserpart=(rpart(SalePrice~.,data=housetraindf[-1]))

In [ ]:
summary(houserpart)

In [ ]:
library(rpart.plot)

In [ ]:
rpart.plot(houserpart,cex=0.40)

In [ ]:
rpartpredict=predict(houserpart,type="vector")

In [ ]:
rpartresidual=housetraindf$SalePrice-rpartpredict

In [ ]:
sqrt(mean(rpartresidual^2))

In [ ]:
library(randomForest)

In [ ]:
houseRF=randomForest(SalePrice~.,data=housetraindf[,-c(1,76,77, 78)],ntree=2000,do.trace=100)

In [ ]:
print(houseRF)

In [ ]:
sqrt(792586627)

In [ ]:
plot(houseRF)

In [ ]:
varImpPlot(houseRF)

In [ ]:
library(gbm)

In [ ]:
housegbm=gbm(SalePrice~.,data=housetraindf[-1],n.trees=2000,distribution = "gaussian",cv.folds=3)

In [ ]:
print(housegbm)

In [ ]:
gbmresidual=housetraindf$SalePrice-housegbm$cv.fitted

In [ ]:
sqrt(mean(gbmresidual^2))

In [ ]:
summary(housegbm)

In [ ]:
library(e1071)
#SVM

In [ ]:
housesvm=svm(SalePrice~.,data=housetraindf[-1])

In [ ]:
svmresidual=housetraindf$SalePrice-housesvm$fitted

In [ ]:
treetestpredict=predict(houserpart,housetestdf[-1])

In [ ]:
write.csv(treetestpredict,"tress.csv")